# Decision Tree
Link: https://www.analyticsvidhya.com/blog/2021/08/decision-tree-algorithm/

## How to choose the root node
For each categorical features, we identify the feature values (Y & N) into two leaves. For each leaves, we identify the target distribution

## Prediction Model 1: Entropy
Def: Entropy is an information theory metric that measures the impurity or uncertainty in a group of observations. 
- Bigger = impure
- Smaller = pure
- Range: 0-1

Equation: $E= \sum p(x)log(\frac{1}{p(x)}) = -\sum p(x)\log_2(p(x))$

## Prediction Model 2: Gini Impurity
Def: tells us what is the probability of misclassifying an observation. Note that the lower the Gini the better the split and the lower the likelihood of misclassification. <br>
Gini Impurity for a Leaf = 1 - [\(probability of 'Yes'\)$^2$ + \(probability of 'No'\)$^2$]<br>
$Gini=1 - \sum\limits_{i-1}^{n}(p_i)^2$, for each leaves <br>
Then, sum each Gini with its owned weight(number on its leaves / total number) $\sum\limits_{i=1}^{n}n_i/N*Gini_i$

## Prediction Model: Information Gain
Information gain as a measure of how much information a feature provides about a class. Information gain helps to determine the **order of attributes (the higher the first to be the node)** in the nodes of a decision tree. <br>
E(Parent): the impurity of the y without split by features <br>
E(Parent|Feature X1): the impurity of the y after split by features X1 <br>
**Information Gain = E(Parent) - E(Parent|Feature X1)** <br>

### Gini Impurity (Numeric Data)
**Steps**
1. Sort the feature values
2. Calculate all the average value for all adjacent values. For example, [1, 3, 4, 6, 9, 13], average value: [2, 3.5, 5, 7.5, 11]
3. For each average value, we use it for the classifier threshold.
4. Calculate the Gini impurity(Categorical Data)
5. Choose the threshold with the lowest Gini Impurity <br>

After choosing the root node, each for internal nodes (branches), we can choose the next node based on the same Gini impurity method aboved until Gini impurity 0. But there may be **overfit**, so we can 
1. Pruning method
2. the distribution of extreme enough, eg: 3:7 or 2:8 to stop the tree.
3. stop the tree if people in the leave is small enough in a certain level 

In [85]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math
import warnings
warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True)
sns.set(rc={'figure.figsize':(10,8)})
from sklearn.datasets import load_iris

In [86]:
X, y = load_iris(return_X_y=True)

In [9]:
def Convert_num_cat(X, y):
    pass

In [10]:
def split(X, y, threshold):
    class1 = y[np.where(X[:, 0][X[:, 0]>6])]
    class2 = y[np.where(X[:, 0][X[:, 0]<=6])]
    return class1, class2

In [11]:
groups = split(X, y, 2)
entropy = 0
total_num = np.sum([group.shape[0] for group in groups])
for group in groups:
    y = group
    group_size = y.shape[0]
    _, count = np.unique(y, return_counts=True)
    p = count / group_size
    weight = group_size / total_num
    entropy += (p @ np.log(1/p)) * weight
entropy

0.5986259325205303

### Prediction model: Entropy

In [12]:
def Entropy(*groups):
    entropy = 0
    total_num = np.sum([group.shape[0] for group in groups])
    
    for group in groups:
        y = group[1]
        group_size = y.shape[0]
        _, count = np.unique(y, return_counts=True)
        p = count / group_size
        weight = group_size / total_num
        
        entropy += (p @ np.log(1/p)) * weight
    return entropy

### Prediction model 2: Gini impurity

In [13]:
def Gini(*gorups):
    gini = 0
    total_num = np.sum([group.shape[0] for group in groups])
    
    for group in groups:
        y = group[1]
        group_size = y.shape[0]
        _, count = np.unique(y, return_counts=True)
        p = count / group_size
        weight = group_size / total_num
        
        gini += (1 - np.sum(p**2)) * weight
    return entropy

### Information Gain

In [14]:
def Information_Gain(X, y, feature_entropy):
    target, counts = np.unique(y, return_counts=True)
    p = counts / counts.sum()
    parent_entropy = p @ np.log(1/p)
    return parent_entropy - feature_entropy

### Prediction model: Entropy ----------------

In [91]:
X, y = load_iris(return_X_y=True)

In [173]:
fr=open('lense.txt', 'r')
lenses = [inst.strip().split('\t') for inst in fr.readlines()]
lensesLabels = ['age', 'prescript', 'astigmatic', 'tearRate']
X = np.array(lenses)[:, :-1]
y = np.array(lenses)[:, -1]
y

array(['no lenses', 'soft', 'no lenses', 'hard', 'no lenses', 'soft',
       'no lenses', 'hard', 'no lenses', 'soft', 'no lenses', 'hard',
       'no lenses', 'soft', 'no lenses', 'no lenses', 'no lenses',
       'no lenses', 'no lenses', 'hard', 'no lenses', 'soft', 'no lenses',
       'no lenses'], dtype='<U10')

In [169]:
def Entropy(y):
    _, count = np.unique(y, return_counts=True)
    p = count / len(y)
    entropy = p @ -np.log(p)
    return entropy

In [174]:
def Split(X, y, col_index, value):
    sub_group = X[X[:, col_index] == value]
    # find the index of X and assign to y, return corresponding y
    index = np.where(X[:, col_index] == value)[0]
    return y[index]

In [175]:
Entropy(Split(X, y, 0, 'young'))

1.0397207708399179

In [41]:
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1      #the last column is used for the labels
    baseEntropy = entropy(dataSet)
    bestInfoGain = 0.0; bestFeature = -1
    for i in range(numFeatures):        #iterate over all the features
        featList = [example[i] for example in dataSet]#create a list of all the examples of this feature
        uniqueVals = set(featList)       #get a set of unique values
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * entropy(subDataSet)     
        infoGain = baseEntropy - newEntropy     #calculate the info gain; ie reduction in entropy
        if (infoGain > bestInfoGain):       #compare this to the best gain so far
            bestInfoGain = infoGain         #if better than current best, set to best
            bestFeature = i
    return bestFeature   

In [42]:
def majorityCnt(classList):
    classCount={}
    for vote in classList:
        if vote not in classCount.keys(): 
            classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.iteritems(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]

In [57]:
def createTree(dataSet,labels):
    dataSet, labels = createDataSet()
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList):
        print(classList[0])
    if len(dataSet[0]) == 1:
        print(majorityCnt(classList))
    bestFeat = chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    print(labels, bestFeat, bestFeatLabel)
    myTree = {bestFeatLabel:{}}
    print(myTree)
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    print(featValues)
    uniqueVals = set(featValues)
    print(uniqueVals)
    for value in uniqueVals:
        subLabels = labels[:]
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value),subLabels)
        print(myTree)

['no surfacing', 'flippers'] 0 no surfacing
{'no surfacing': {}}
[1, 1, 1, 0, 0]
{0, 1}


NameError: name 'createTree' is not defined